In [1]:
import bd
import pickle
import numpy as np
from Point import Point
from Limb import Limb

import matplotlib
%matplotlib inline
from matplotlib import pyplot as plt

In [2]:
#constants
ratio = 2
black = (0,0,0)
red = (1,0,0)
width, height = 1000, 1000

#to keep
l_arm = True
r_arm = True
l_leg = True
r_leg = True
neck = True
general = True

to_keep = [l_arm, r_arm, l_leg, r_leg, neck, general]

In [ ]:
myPaintings = pickle.load(open('../../Python Scripts/myPaintings21400.p', 'rb'))


## HERE go buc corrections to the catalog, which are numerous.... 
myPaintings[7465][1] = 'http://www.wga.hu/art/c/casa/portrait_C.jpg' ## and NOT portrait_c (lowercase), the original...


In [ ]:
#6 seconds in my computer
all_bodies = bd.construct_body_list(21400, myPaintings)

In [ ]:
all_relative_angles = bd.all_relative_angles(all_bodies)
deviation = bd.compute_std_deviation(all_relative_angles)

In [ ]:
ll = bd.all_bodies_mean_limb_length(all_bodies)  * np.array(ratio)
ml = bd.middle_mean_length(all_bodies) * ratio

In [ ]:
resized_bodies = bd.resize_bodies(all_bodies, ml)
mlb = bd.middle_mean_length(resized_bodies)

In [ ]:
body_id = 52500
img = np.ones((height, width, 3))


resized_bodies[body_id].draw(img, red)


plt.imshow(img)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(10, 10)

In [ ]:
img = np.ones((height, width, 3))
base_body = bd.angles_to_body(bd.angles(), ll, Point(500,300))

base_body.rotate(30)
base_body.draw(img, red)


plt.imshow(img)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(10, 10)

In [ ]:
d,i = bd.get_n_nearest_neighbor(all_relative_angles, base_body, deviation, to_keep, n=100, dist=50)
print i

In [ ]:
#to change the number of random paintings, change n
n = 10
#bd.p_values_random_hypothesis(n, myPaintings, all_bodies, all_relative_angles, deviation, 'a')

In [ ]:
def interactiive_body(body, angles, bodies, r_arm = True, l_arm = True, r_leg = True, l_leg = True, neck_p = True):
    """Plot an interactive body with which we can play. Only call this function on a complete body (for now)"""
    def refresh(_):
        i = 0
        to_update = []
        if l_arm:
            left_arm.x, left_arm.y = [[p[2][0], scat.x[i], scat.x[i+1]],[p[2][1], scat.y[i], scat.y[i+1]]]
            to_update.append(3)
            to_update.append(4)
            i+=2
        if r_arm:
            right_arm.x, right_arm.y = [[p[6][0], scat.x[i], scat.x[i+1]],[p[6][1], scat.y[i], scat.y[i+1]]]
            to_update.append(7)
            to_update.append(8)
            i+=2
        if l_leg:
            left_leg.x, left_leg.y = [[p[9][0], scat.x[i], scat.x[i+1]],[p[9][1], scat.y[i], scat.y[i+1]]]
            to_update.append(10)
            to_update.append(11)
            i+=2
        if r_leg:
            right_leg.x, right_leg.y = [[p[13][0], scat.x[i], scat.x[i+1]],[p[13][1], scat.y[i], scat.y[i+1]]]
            to_update.append(14)
            to_update.append(15)
            i+=2
        if neck_p:
            to_update.append(0)
            neck.x, neck.y = [[p[1][0], scat.x[i]],[p[1][1], scat.y[i]]]
        
        head.x = np.cos(np.linspace(0, 2*np.pi, 100))*60+neck.x[1]
        head.y = np.sin(np.linspace(0, 2*np.pi, 100))*65+neck.y[1]
        
        #update body
        for i in range(len(to_update)):
            body[0][to_update[i]] = [scat.x[i], scat.y[i]]
        
        body_angles = []
        b_angles = member_relative_angles(body)
        
        if l_arm:
            body_angles.append(b_angles[0])
            body_angles.append(b_angles[1])
        if r_arm:
            body_angles.append(b_angles[2])
            body_angles.append(b_angles[3])
        if l_leg:
            body_angles.append(b_angles[4])
            body_angles.append(b_angles[5])
        if r_leg:
            body_angles.append(b_angles[6])
            body_angles.append(b_angles[7])
        if neck_p:
            body_angles.append(b_angles[8])
        
        
        b =  get_nearest_neighbor(np.transpose(all_angles), np.transpose(body_angles), bodies)[0]
        p2 = b[0]
        nchest.x, nchest.y = np.transpose([p2[1], p2[2], p2[9], p2[13], p2[6], p2[1], p2[0]])
        
        nhead.x = np.cos(np.linspace(0, 2*np.pi, 100))*60+p2[0][0]
        nhead.y = np.sin(np.linspace(0, 2*np.pi, 100))*65+p2[0][1]
        
        nleft_arm.x, nleft_arm.y = [[p2[2][0], p2[3][0], p2[4][0]],[p2[2][1], p2[3][1], p2[4][1]]]
        nright_arm.x, nright_arm.y = [[p2[6][0], p2[7][0], p2[8][0]],[p2[6][1], p2[7][1], p2[8][1]]]
        nleft_leg.x, nleft_leg.y = [[p2[9][0], p2[10][0], p2[11][0]],[p2[9][1], p2[10][1], p2[11][1]]]
        nright_leg.x, nright_leg.y = [[p2[13][0], p2[14][0], p2[15][0]],[p2[13][1], p2[14][1], p2[15][1]]]
        
        
        
    
    scales = {'x': bqp.LinearScale(min= 0, max= 1000),
             'y' : bqp.LinearScale(min = 1000, max = 0)}
    
    #initialization of the nearest neighbor of the interactive body.
    body_angles = []
    all_angles = []
    b_angles = member_relative_angles(body)
    a = np.transpose(angles)
    
    if l_arm:
        body_angles.append(b_angles[0])
        body_angles.append(b_angles[1])
        all_angles.append(a[0])
        all_angles.append(a[1])
    if r_arm:
        body_angles.append(b_angles[2])
        body_angles.append(b_angles[3])
        all_angles.append(a[2])
        all_angles.append(a[3])
    if l_leg:
        body_angles.append(b_angles[4])
        body_angles.append(b_angles[5])
        all_angles.append(a[4])
        all_angles.append(a[5])
    if r_leg:
        body_angles.append(b_angles[6])
        body_angles.append(b_angles[7])
        all_angles.append(a[6])
        all_angles.append(a[7])
    if neck_p:
        body_angles.append(b_angles[8])
        all_angles.append(a[8])
        
    nbody = get_nearest_neighbor(np.transpose(all_angles), np.transpose(body_angles), bodies)[0]
    
    #points of the interactive and neighbor bodies
    p = body[0]
    p2 = nbody[0]
    
    marks = []
    #Constructions of the two bodies: the interactive one and its nearest neighbor. body part beginnig with n... are part of the neighbor.
    #draw the chest
    chest = bqp.Lines(scales=scales)
    chest.x, chest.y = np.transpose([p[1], p[2], p[9], p[13], p[6], p[1]])
    marks.append(chest)
    
    nchest = bqp.Lines(scales=scales,  colors=['red'])
    nchest.x, nchest.y = np.transpose([p2[1], p2[2], p2[9], p2[13], p2[6], p2[1], p2[0]])
    marks.append(nchest)
    
    #draw the head
    head = bqp.Lines(scales=scales)
    head.x = np.cos(np.linspace(0, 2*np.pi, 100))*60+p[0][0]
    head.y = np.sin(np.linspace(0, 2*np.pi, 100))*65+p[0][1]
    marks.append(head)
    
    
    nhead_x = np.cos(np.linspace(0, 2*np.pi, 100))*60+p2[0][0]
    nhead_y = np.sin(np.linspace(0, 2*np.pi, 100))*65+p2[0][1]
    nhead = bqp.Lines(x=nhead_x, y=nhead_y, scales=scales,  colors=['red'])
    marks.append(nhead)
    
    to_keep=list()
    if l_arm:
        to_keep.append(p[3])
        to_keep.append(p[4])
    if r_arm:
        to_keep.append(p[7])
        to_keep.append(p[8])
    if l_leg:
        to_keep.append(p[10])
        to_keep.append(p[11])
    if r_leg:
        to_keep.append(p[14])
        to_keep.append(p[15])
    if neck_p:
        to_keep.append(p[0])
    #movable points: arms first, left side first
    scat = bqp.Scatter(scales = scales, enable_move = True, update_on_move = True, stroke_width = 7)
    scat.x , scat.y = np.transpose(to_keep)
    marks.append(scat)
    
    i = 0
    left_arm = bqp.Lines(scales=scales)
    if l_arm:
        left_arm.x, left_arm.y = [[p[2][0], scat.x[i], scat.x[i+1]],[p[2][1], scat.y[i], scat.y[i+1]]]
        i +=2
    else:
        left_arm.x, left_arm.y = [[p[2][0], p[3][0], p[4][0]],[p[2][1], p[3][1],p[4][1]]]
    marks.append(left_arm)
    
    nleft_arm = bqp.Lines(scales=scales, colors=['red'])
    nleft_arm.x, nleft_arm.y = [[p2[2][0], p2[3][0], p2[4][0]],[p2[2][1], p2[3][1], p2[4][1]]]
    marks.append(nleft_arm)
    
    right_arm = bqp.Lines(scales=scales)
    if r_arm:
        right_arm.x, right_arm.y = [[p[6][0], scat.x[i], scat.x[i+1]],[p[6][1], scat.y[i], scat.y[i+1]]]
        i+=2
    else:
        right_arm.x, right_arm.y = [[p[6][0], p[7][0], p[8][0]],[p[6][1], p[7][1],p[8][1]]]
    
    marks.append(right_arm)
    
    nright_arm = bqp.Lines(scales=scales,  colors=['red'])
    nright_arm.x, nright_arm.y = [[p2[6][0], p2[7][0], p2[8][0]],[p2[6][1], p2[7][1], p2[8][1]]]
    marks.append(nright_arm)
    
    
    left_leg = bqp.Lines(scales=scales)
    if l_leg:
        left_leg.x, left_leg.y = [[p[9][0], scat.x[i], scat.x[i+1]],[p[9][1], scat.y[i], scat.y[i+1]]]
        i+=2
    else:
        left_leg.x, left_leg.y = [[p[9][0], p[10][0], p[11][0]],[p[9][1], p[10][1],p[11][1]]]
    marks.append(left_leg)
    
    nleft_leg = bqp.Lines(scales=scales,  colors=['red'])
    nleft_leg.x, nleft_leg.y = [[p2[9][0], p2[10][0], p2[11][0]],[p2[9][1], p2[10][1], p2[11][1]]]
    marks.append(nleft_leg)
    
    
    right_leg = bqp.Lines(scales=scales)
    if r_leg:
        right_leg.x, right_leg.y = [[p[13][0], scat.x[i], scat.x[i+1]],[p[13][1], scat.y[i], scat.y[i+1]]]
        i+=2
    else:
        right_leg.x, right_leg.y = [[p[13][0], p[14][0], p[15][0]],[p[13][1], p[14][1], p[15][1]]]
    marks.append(right_leg)
    
    
    nright_leg = bqp.Lines(scales=scales,  colors=['red'])
    nright_leg.x, nright_leg.y = [[p2[13][0], p2[14][0], p2[15][0]],[p2[13][1], p2[14][1], p2[15][1]]]
    marks.append(nright_leg)
    
    
    neck =  bqp.Lines(scales=scales)
    if neck_p:
        neck.x, neck.y = [[p[1][0], scat.x[i]],[p[1][1], scat.y[i]]]
        i+=1
    else:
        neck.x, neck.y = [[p[1][0], p[0][0]],[p[1][1], p[0][1]]]
    marks.append(neck)
    scat.observe(refresh, names=['x', 'y'])
    

    return bqp.Figure(marks=marks, 
                      padding_y = 0., min_height = 750, min_width = 750)

In [ ]:
import bqplot as bqp


def interactive_body(body, l_arm = True, r_arm = True, l_leg = True, r_leg = True, neck_p = True, general = True):
    """Plot an interactive body with which we can play."""
    def refresh(_):
        middle = Limb(body.pts[9], body.pts[13]).middle()
        if (not abs(scat.x[-1] - middle.x) < 1) or (not abs(scat.y[-1] - middle.y)<1):
            o = body.pts[1]
            a = Point(scat.x[-1], scat.y[-1]).angle(o) - middle.angle(o)
            print a
            body.rotate(a)
            
            middle = Limb(body.pts[9], body.pts[13]).middle()
            keep=list()
            if l_arm:
                keep.append(body.pts[3].to_array())
                keep.append(body.pts[4].to_array())
            if r_arm:
                keep.append(body.pts[7].to_array())
                keep.append(body.pts[8].to_array())
            if l_leg:
                keep.append(body.pts[10].to_array())
                keep.append(body.pts[11].to_array())
            if r_leg:
                keep.append(body.pts[14].to_array())
                keep.append(body.pts[15].to_array())
            if neck_p:
                keep.append(body.pts[0].to_array())
            if general:
                keep.append(middle.to_array())

            #movable points: arms first, left side first
            scat.x , scat.y = np.transpose(keep)
        
        chest.x, chest.y = np.transpose([body.pts[1].to_array(), body.pts[2].to_array(), body.pts[9].to_array(), \
                                     body.pts[13].to_array(), body.pts[6].to_array(), body.pts[1].to_array()])
        
        i = 0
        to_update = []
        if l_arm:
            left_arm.x, left_arm.y = [[body.pts[2].x, scat.x[i], scat.x[i+1]],[body.pts[2].y, scat.y[i], scat.y[i+1]]]
            to_update.append(3)
            to_update.append(4)
            i+=2
        if r_arm:
            right_arm.x, right_arm.y = [[body.pts[6].x, scat.x[i], scat.x[i+1]],[body.pts[6].y, scat.y[i], scat.y[i+1]]]
            to_update.append(7)
            to_update.append(8)
            i+=2
        if l_leg:
            left_leg.x, left_leg.y = [[body.pts[9].x, scat.x[i], scat.x[i+1]],[body.pts[9].y, scat.y[i], scat.y[i+1]]]
            to_update.append(10)
            to_update.append(11)
            i+=2
        if r_leg:
            right_leg.x, right_leg.y = [[body.pts[13].x, scat.x[i], scat.x[i+1]],[body.pts[13].y, scat.y[i], scat.y[i+1]]]
            to_update.append(14)
            to_update.append(15)
            i+=2
        if neck_p:
            to_update.append(0)
            neck.x, neck.y = [[body.pts[1].x, scat.x[i]],[body.pts[1].y, scat.y[i]]]
            
        
        head.x = np.cos(np.linspace(0, 2*np.pi, 100))*60+body.pts[0].x
        head.y = np.sin(np.linspace(0, 2*np.pi, 100))*65+body.pts[0].y
        
        
        #update body
        for i in range(len(to_update)):
            body.pts[to_update[i]].x = scat.x[i]
            body.pts[to_update[i]].y =  scat.y[i]
        
        return
    
    
    
    scales = {'x': bqp.LinearScale(min= 0, max= 1000),
             'y' : bqp.LinearScale(min = 1000, max = 0)}
    
    marks = []
    
    middle = Limb(body.pts[9], body.pts[13]).middle()
    to_keep=list()
    if l_arm:
        to_keep.append(body.pts[3].to_array())
        to_keep.append(body.pts[4].to_array())
    if r_arm:
        to_keep.append(body.pts[7].to_array())
        to_keep.append(body.pts[8].to_array())
    if l_leg:
        to_keep.append(body.pts[10].to_array())
        to_keep.append(body.pts[11].to_array())
    if r_leg:
        to_keep.append(body.pts[14].to_array())
        to_keep.append(body.pts[15].to_array())
    if neck_p:
        to_keep.append(body.pts[0].to_array())
    if general:
        to_keep.append(middle.to_array())
    
    #movable points: arms first, left side first
    scat = bqp.Scatter(scales = scales, enable_move = True, update_on_move = True, stroke_width = 7)
    scat.x , scat.y = np.transpose(to_keep)
    marks.append(scat)
    
    #draw the chest
    chest = bqp.Lines(scales=scales)
    chest.x, chest.y = np.transpose([body.pts[1].to_array(), body.pts[2].to_array(), body.pts[9].to_array(), \
                                     body.pts[13].to_array(), body.pts[6].to_array(), body.pts[1].to_array()])
    marks.append(chest)
    
    
    #draw the head
    head = bqp.Lines(scales=scales)
    head.x = np.cos(np.linspace(0, 2*np.pi, 100))*60+body.pts[0].x
    head.y = np.sin(np.linspace(0, 2*np.pi, 100))*65+body.pts[0].y
    marks.append(head)
    
    i = 0
    #draw the left arm
    left_arm = bqp.Lines(scales=scales)
    if l_arm:
        left_arm.x, left_arm.y = [[body.pts[2].x, scat.x[i], scat.x[i+1]],[body.pts[2].y, scat.y[i], scat.y[i+1]]]
        i +=2
    else:
        left_arm.x, left_arm.y = [[body.pts[2].x, body.pts[3].x, body.pts[4].x],[body.pts[2].y, body.pts[3].y, body.pts[4].y]]
    marks.append(left_arm)
    
    
    #draw the right arm
    right_arm = bqp.Lines(scales=scales)
    if r_arm:
        right_arm.x, right_arm.y = [[body.pts[6].x, scat.x[i], scat.x[i+1]],[body.pts[6].y, scat.y[i], scat.y[i+1]]]
        i+=2
    else:
        right_arm.x, right_arm.y = [[body.pts[6].x, body.pts[7].x, body.pts[8].x],[body.pts[6].y, body.pts[7].y,body.pts[8].y]]
    
    marks.append(right_arm)
    
    
    #draw the left leg
    left_leg = bqp.Lines(scales=scales)
    if l_leg:
        left_leg.x, left_leg.y = [[body.pts[9].x, scat.x[i], scat.x[i+1]],[body.pts[9].y, scat.y[i], scat.y[i+1]]]
        i+=2
    else:
        left_leg.x, left_leg.y = [[body.pts[9].x, body.pts[10].x, body.pts[11].x],[body.pts[9].y, body.pts[10].y,body.pts[11].y]]
    marks.append(left_leg)
    
    
    #draw the right leg
    right_leg = bqp.Lines(scales=scales)
    if r_leg:
        right_leg.x, right_leg.y = [[body.pts[13].x, scat.x[i], scat.x[i+1]],[body.pts[13].y, scat.y[i], scat.y[i+1]]]
        i+=2
    else:
        right_leg.x, right_leg.y = [[body.pts[13].x, body.pts[14].x, body.pts[15].x],[body.pts[13].y, body.pts[14].y, body.pts[15].y]]
    marks.append(right_leg)
    
    
    #draw the neck
    neck =  bqp.Lines(scales=scales)
    if neck_p:
        neck.x, neck.y = [[body.pts[1].x, scat.x[i]],[body.pts[1].y, scat.y[i]]]
        i+=1
    else:
        neck.x, neck.y = [[body.pts[1].x, body.pts[0].x],[body.pts[1].y, body.pts[0].y]]
    marks.append(neck)
    
    scat.observe(refresh, names=['x', 'y'])
    
    return bqp.Figure(marks=marks, padding_y = 0., min_height = 750, min_width = 750)


base_body = bd.angles_to_body(bd.angles(), ll, Point(500,300))
interactive_body(base_body)